In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *


fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/boxscores_team/"

In [ ]:
season = "2023"
df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
df['Team_Split'] = df['MATCHUP'].str.replace("@","")
df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
df[["FG3M","PLUS_MINUS","FG3_PCT"]] = df[["FG3M","PLUS_MINUS","FG3_PCT"]].astype(str)

In [ ]:
df.columns

# Based on 3FGM

In [ ]:
df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3M","PLUS_MINUS"]].agg(",".join).reset_index()
df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
df1[["FG3M_For", "FG3M_Against"]] = df1["FG3M"].str.split(",", expand=True)
df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3M","PLUS_MINUS"]].agg(",".join).reset_index()
df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
df2[["FG3M_Against","FG3M_For"]] = df2["FG3M"].str.split(",", expand=True)
df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)

In [ ]:
df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
df3 = df3[["Team1","Team2","FG3M_For", "FG3M_Against","PM1","PM2"]]
df3.iloc[:,2:] = df3.iloc[:,2:].astype(int)

In [ ]:
df3["More_3PT"] = df3["FG3M_For"] > df3["FG3M_Against"]
df3["Less_3PT"] = df3["FG3M_For"] <= df3["FG3M_Against"]
df3["Win"] = df3["PM1"] > 0
df3["Loss"] = df3["PM1"] < 0
df3["Win_More_3PT"] = df3["More_3PT"] & df3["Win"]
df3["Win_Less_3PT"] = df3["Less_3PT"] & df3["Win"]
df3["Loss_More_3PT"] = df3["More_3PT"] & df3["Loss"]
df3["Loss_Less_3PT"] = df3["Less_3PT"] & df3["Loss"]

In [ ]:
df4 = df3.groupby("Team1")[["Win_More_3PT","Win_Less_3PT","Loss_More_3PT","Loss_Less_3PT"]]\
.agg({"Win_More_3PT":["sum"],"Win_Less_3PT":["sum"],"Loss_More_3PT":["sum"],"Loss_Less_3PT":["sum"]})
df4.columns = ["Win_More_3PT","Win_Less_3PT","Loss_More_3PT","Loss_Less_3PT"]
df4 = df4.sort_values(by="Win_More_3PT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
df4.loc['Total']= df4.sum()
df4.loc['Total','Team'] = "Total"

In [ ]:
d = (
    gt.GT(data=df4.iloc[:15])
    # .cols_label(For_30= "30 PTS For", Against_30="30 PTS Against", Diff_30="Difference")
    .cols_align(align="center", columns=["Win_More_3PT","Win_Less_3PT","Loss_More_3PT","Loss_Less_3PT"])
    .tab_spanner(
        label = "More 3PT",
        columns = ["Win_More_3PT", "Loss_More_3PT"]
    )
    .tab_spanner(
        label = "Less 3PT",
        columns = ["Win_Less_3PT", "Loss_Less_3PT"]
    )
    .tab_header(
        title="Team Win Loss When Making More 3 Pointers than Opponent",
    )
    .cols_label(Win_More_3PT = "Wins",Win_Less_3PT = "Wins", Loss_More_3PT = "Losses",Loss_Less_3PT = "Losses")
    .tab_source_note(
        source_note = "@SravanNBA | source: nba.com/stats"
    )
)
d

In [ ]:
hti.screenshot(html_str=d._render_as_html(), save_as="test1.png", size=(600, 750))

In [ ]:
d = (
    gt.GT(data=df4.iloc[15:])
    # .cols_label(For_30= "30 PTS For", Against_30="30 PTS Against", Diff_30="Difference")
    .cols_align(align="center", columns=["Win_More_3PT","Win_Less_3PT","Loss_More_3PT","Loss_Less_3PT"])
    .tab_spanner(
        label = "More 3PT",
        columns = ["Win_More_3PT", "Loss_More_3PT"]
    )
    .tab_spanner(
        label = "Less 3PT",
        columns = ["Win_Less_3PT", "Loss_Less_3PT"]
    )
    .tab_header(
        title="Team Win Loss When Making More 3 Pointers than Opponent",
    )
    .cols_label(Win_More_3PT = "Wins",Win_Less_3PT = "Wins", Loss_More_3PT = "Losses",Loss_Less_3PT = "Losses")
    .tab_source_note(
        source_note = "@SravanNBA | source: nba.com/stats"
    )
)
d

In [ ]:
hti.screenshot(html_str=d._render_as_html(), save_as="test2.png", size=(600, 750))

# Based on 3PT%

In [ ]:
df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3_PCT","PLUS_MINUS"]].agg(",".join).reset_index()
df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
df1[["FG3_PCT_For", "FG3_PCT_Against"]] = df1["FG3_PCT"].str.split(",", expand=True)
df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3_PCT","PLUS_MINUS"]].agg(",".join).reset_index()
df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
df2[["FG3_PCT_Against","FG3_PCT_For"]] = df2["FG3_PCT"].str.split(",", expand=True)
df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)

In [ ]:
df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
df3 = df3[["Team1","Team2","FG3_PCT_For", "FG3_PCT_Against","PM1","PM2"]]
df3.iloc[:,2:] = df3.iloc[:,2:].astype(float)

In [ ]:
df3["More_3PT_PCT"] = df3["FG3_PCT_For"] > (df3["FG3_PCT_Against"] +0.1)
df3["Win"] = df3["PM1"] > 0
df3["Loss"] = df3["PM1"] < 0
df3["Win_More_3PT_PCT"] = df3["More_3PT_PCT"] & df3["Win"]
df3["Loss_More_3PT_PCT"] = df3["More_3PT_PCT"] & df3["Loss"]

In [ ]:
df4 = df3.groupby("Team1")[["Win_More_3PT_PCT","Loss_More_3PT_PCT"]]\
.agg({"Win_More_3PT_PCT":["sum"],"Loss_More_3PT_PCT":["sum"]})
df4.columns = ["Win_More_3PT_PCT","Loss_More_3PT_PCT"]
df4 = df4.sort_values(by="Win_More_3PT_PCT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
df4.loc['Total']= df4.sum()
df4.loc['Total','Team'] = "Total"

In [ ]:
d = (
    gt.GT(data=df4)
    # .cols_label(For_30= "30 PTS For", Against_30="30 PTS Against", Diff_30="Difference")
    .cols_align(align="center", columns=["Win_More_3PT_PCT","Loss_More_3PT_PCT"])
    .tab_spanner(
        label = "3PT % > Opponent by 10%",
        columns = ["Win_More_3PT_PCT", "Loss_More_3PT_PCT"]
    )
    .tab_header(
        title="Team Win Loss When 3PT% greater than Opponent by 10%",
    )
    .cols_label(Win_More_3PT_PCT = "Wins",Loss_More_3PT_PCT = "Losses")
    .tab_source_note(
        source_note = "@SravanNBA | source: nba.com/stats"
    )
)
d

In [ ]:
hti.screenshot(html_str=d._render_as_html(), save_as="test3.png", size=(400, 1400))

# 3PT Shooting Year to Year Trends

In [ ]:
from nba_api.stats.endpoints import leaguedashteamstats


In [ ]:
dfa = []
for year in range(2014,2024):
    season = str(year) + "-" + str(year + 1)[-2:]
    print(season)
    stats = leaguedashteamstats.LeagueDashTeamStats(per_mode_detailed="PerGame", season_type_all_star="Regular Season", season = season)
    df = stats.get_data_frames()[0]
    df = df[["TEAM_NAME","W","L","W_PCT","FG3M","FG3A","FG3_PCT"]]
    df.loc["Total"] = df.sum()
    df.loc["Total","FG3M"] = df.loc["Total","FG3M"]/30
    df.loc["Total","FG3A"] = df.loc["Total","FG3A"]/30 
    df.loc["Total","FG3_PCT"] = df.loc["Total","FG3M"]/df.loc["Total","FG3A"]
    df.loc["Total","TEAM_NAME"] = year +1
    df.iloc[:,4:] = df.iloc[:,4:].round(3)
    dft = df.tail(1)
    dft = dft[["TEAM_NAME","FG3M", "FG3A", "FG3_PCT"]]
    dfa.append(dft)
df_3pct = pd.concat(dfa)

In [ ]:
df_3pct = df_3pct.rename(columns={"TEAM_NAME":"Season"}).reset_index(drop=True)

In [ ]:
df_3pct

In [ ]:
theme_idv = themes.theme_xkcd(base_size=11)
theme_idv += theme(
    plot_title=element_text(face="bold", size=16),
)

In [ ]:
p = (
    ggplot(df_3pct)
    + aes(x="Season", y= "FG3A")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "3PT Attempts per Game 2014-2023",
            caption="@SravanNBA | source:nba.com/stats",
        )
)
p.save(fig_DIR + "FG3A_seasons.png", width=5, height=4, dpi=300)
p.draw()

In [ ]:
p = (
    ggplot(df_3pct)
    + aes(x="Season", y= "FG3M")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "3PT Makes per Game 2014-2023",
            caption="@SravanNBA | source:nba.com/stats",
        )
)
p.save(fig_DIR + "FG3M_seasons.png", width=5, height=4, dpi=300)
p.draw()

In [ ]:
p = (
    ggplot(df_3pct)
    + aes(x="Season", y= "FG3_PCT")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "3PT Accuracy 2014-2023",
            caption="@SravanNBA | source:nba.com/stats",
            y="3PT Percent"
        )
)
p.save(fig_DIR + "FG3_PCT_seasons.png", width=5, height=4, dpi=300)
p.draw()

# 3FGA vs Wins Year to Year

### Team 3PA > Opponent 3PA

In [ ]:
dfa = []
for season in range(2014,2024):
    df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df['Team_Split'] = df['MATCHUP'].str.replace("@","")
    df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
    df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
    df[["FG3A","PLUS_MINUS","FG3_PCT"]] = df[["FG3A","PLUS_MINUS","FG3_PCT"]].astype(str)
    df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3A","PLUS_MINUS"]].agg(",".join).reset_index()
    df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df1[["FG3A_For", "FG3A_Against"]] = df1["FG3A"].str.split(",", expand=True)
    df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
    df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3A","PLUS_MINUS"]].agg(",".join).reset_index()
    df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df2[["FG3A_Against","FG3A_For"]] = df2["FG3A"].str.split(",", expand=True)
    df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)
    df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
    df3 = df3[["Team1","Team2","FG3A_For", "FG3A_Against","PM1","PM2"]]
    df3.iloc[:,2:] = df3.iloc[:,2:].astype(int)
    df3["More_3PT"] = df3["FG3A_For"] > (df3["FG3A_Against"] + 0)
    df3["Win"] = df3["PM1"] > 0
    df3["Loss"] = df3["PM1"] < 0
    df3["Win_More_3PT"] = df3["More_3PT"] & df3["Win"]
    df3["Loss_More_3PT"] = df3["More_3PT"] & df3["Loss"]
    df4 = df3.groupby("Team1")[["Win_More_3PT","Loss_More_3PT"]]\
        .agg({"Win_More_3PT":["sum"],"Loss_More_3PT":["sum"]})
    df4.columns = ["Win_More_3PT","Loss_More_3PT"]
    df4 = df4.sort_values(by="Win_More_3PT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
    df4.loc['Total']= df4.sum()
    df4.loc['Total','Team'] = "Total"
    df4.loc['Total','Team'] = season +1
    dft = df4.tail(1)
    dfa.append(dft)
df_aw = pd.concat(dfa).reset_index(drop=True)
df_aw = df_aw.rename(columns={"Team":"Season"})
df_aw["Win_PCT"] = (df_aw["Win_More_3PT"]/(df_aw["Win_More_3PT"]+df_aw["Loss_More_3PT"])).round(3)

In [ ]:
p = (
    ggplot(df_aw)
    + aes(x="Season", y= "Win_PCT")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "Team Win % When Attempting More 3 Pointers than Opponent",
            caption="@SravanNBA | source:nba.com/stats",
            y = "Win %",
        )
    + theme(plot_title=element_text(face="bold", size=11))
)
p.save(fig_DIR + "FG3A_Wins_seasons.png", width=5, height=4, dpi=300)
p.draw()

### Team 3PA > Opponent 3PA + 5

In [ ]:
dfa = []
for season in range(2014,2024):
    df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df['Team_Split'] = df['MATCHUP'].str.replace("@","")
    df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
    df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
    df[["FG3A","PLUS_MINUS","FG3_PCT"]] = df[["FG3A","PLUS_MINUS","FG3_PCT"]].astype(str)
    df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3A","PLUS_MINUS"]].agg(",".join).reset_index()
    df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df1[["FG3A_For", "FG3A_Against"]] = df1["FG3A"].str.split(",", expand=True)
    df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
    df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3A","PLUS_MINUS"]].agg(",".join).reset_index()
    df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df2[["FG3A_Against","FG3A_For"]] = df2["FG3A"].str.split(",", expand=True)
    df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)
    df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
    df3 = df3[["Team1","Team2","FG3A_For", "FG3A_Against","PM1","PM2"]]
    df3.iloc[:,2:] = df3.iloc[:,2:].astype(int)
    df3["More_3PT"] = df3["FG3A_For"] > (df3["FG3A_Against"] + 0)
    df3["Win"] = df3["PM1"] > 0
    df3["Loss"] = df3["PM1"] < 0
    df3["Win_More_3PT"] = df3["More_3PT"] & df3["Win"]
    df3["Loss_More_3PT"] = df3["More_3PT"] & df3["Loss"]
    df4 = df3.groupby("Team1")[["Win_More_3PT","Loss_More_3PT"]]\
        .agg({"Win_More_3PT":["sum"],"Loss_More_3PT":["sum"]})
    df4.columns = ["Win_More_3PT","Loss_More_3PT"]
    df4 = df4.sort_values(by="Win_More_3PT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
    df4.loc['Total']= df4.sum()
    df4.loc['Total','Team'] = "Total"
    df4.loc['Total','Team'] = season +1
    dft = df4.tail(1)
    dfa.append(dft)
df_aw = pd.concat(dfa).reset_index(drop=True)
df_aw = df_aw.rename(columns={"Team":"Season"})
df_aw["Win_PCT"] = (df_aw["Win_More_3PT"]/(df_aw["Win_More_3PT"]+df_aw["Loss_More_3PT"])).round(3)

In [ ]:
p = (
    ggplot(df_aw)
    + aes(x="Season", y= "Win_PCT")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "Team Win % When Attempting 5 More 3 Pointers than Opponent",
            caption="@SravanNBA | source:nba.com/stats",
            y = "Win %",
        )
    + theme(plot_title=element_text(face="bold", size=11))
)
p.save(fig_DIR + "FG3A_Wins_seasons_1.png", width=5.2, height=4, dpi=300)
p.draw()

# 3FGM vs Wins Year to Year

### Team 3PM > Opponent 3PM

In [ ]:
dfa = []
for season in range(2014,2024):
    df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df['Team_Split'] = df['MATCHUP'].str.replace("@","")
    df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
    df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
    df[["FG3M","PLUS_MINUS","FG3_PCT"]] = df[["FG3M","PLUS_MINUS","FG3_PCT"]].astype(str)
    df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3M","PLUS_MINUS"]].agg(",".join).reset_index()
    df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df1[["FG3M_For", "FG3M_Against"]] = df1["FG3M"].str.split(",", expand=True)
    df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
    df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3M","PLUS_MINUS"]].agg(",".join).reset_index()
    df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df2[["FG3M_Against","FG3M_For"]] = df2["FG3M"].str.split(",", expand=True)
    df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)
    df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
    df3 = df3[["Team1","Team2","FG3M_For", "FG3M_Against","PM1","PM2"]]
    df3.iloc[:,2:] = df3.iloc[:,2:].astype(int)
    df3["More_3PT"] = df3["FG3M_For"] > (df3["FG3M_Against"])
    df3["Win"] = df3["PM1"] > 0
    df3["Loss"] = df3["PM1"] < 0
    df3["Win_More_3PT"] = df3["More_3PT"] & df3["Win"]
    df3["Loss_More_3PT"] = df3["More_3PT"] & df3["Loss"]
    df4 = df3.groupby("Team1")[["Win_More_3PT","Loss_More_3PT"]]\
        .agg({"Win_More_3PT":["sum"],"Loss_More_3PT":["sum"]})
    df4.columns = ["Win_More_3PT","Loss_More_3PT"]
    df4 = df4.sort_values(by="Win_More_3PT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
    df4.loc['Total']= df4.sum()
    df4.loc['Total','Team'] = "Total"
    df4.loc['Total','Team'] = season +1
    dft = df4.tail(1)
    dfa.append(dft)
df_mw = pd.concat(dfa).reset_index(drop=True)
df_mw = df_mw.rename(columns={"Team":"Season"})
df_mw["Win_PCT"] = (df_mw["Win_More_3PT"]/(df_mw["Win_More_3PT"]+df_mw["Loss_More_3PT"])).round(3)

In [ ]:
p = (
    ggplot(df_mw)
    + aes(x="Season", y= "Win_PCT")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "Team Win % When Making More 3 Pointers than Opponent",
            caption="@SravanNBA | source:nba.com/stats",
            y = "Win %",
        )
    + theme(plot_title=element_text(face="bold", size=11.5))
)
p.save(fig_DIR + "FG3M_Wins_seasons.png", width=5, height=4, dpi=300)
p.draw()

### Team 3PM > Opponent 3PM + 2

In [ ]:
dfa = []
for season in range(2014,2024):
    df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df['Team_Split'] = df['MATCHUP'].str.replace("@","")
    df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
    df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
    df[["FG3M","PLUS_MINUS","FG3_PCT"]] = df[["FG3M","PLUS_MINUS","FG3_PCT"]].astype(str)
    df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3M","PLUS_MINUS"]].agg(",".join).reset_index()
    df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df1[["FG3M_For", "FG3M_Against"]] = df1["FG3M"].str.split(",", expand=True)
    df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
    df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3M","PLUS_MINUS"]].agg(",".join).reset_index()
    df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df2[["FG3M_Against","FG3M_For"]] = df2["FG3M"].str.split(",", expand=True)
    df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)
    df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
    df3 = df3[["Team1","Team2","FG3M_For", "FG3M_Against","PM1","PM2"]]
    df3.iloc[:,2:] = df3.iloc[:,2:].astype(int)
    df3["More_3PT"] = df3["FG3M_For"] > (df3["FG3M_Against"] + 2)
    df3["Win"] = df3["PM1"] > 0
    df3["Loss"] = df3["PM1"] < 0
    df3["Win_More_3PT"] = df3["More_3PT"] & df3["Win"]
    df3["Loss_More_3PT"] = df3["More_3PT"] & df3["Loss"]
    df4 = df3.groupby("Team1")[["Win_More_3PT","Loss_More_3PT"]]\
        .agg({"Win_More_3PT":["sum"],"Loss_More_3PT":["sum"]})
    df4.columns = ["Win_More_3PT","Loss_More_3PT"]
    df4 = df4.sort_values(by="Win_More_3PT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
    df4.loc['Total']= df4.sum()
    df4.loc['Total','Team'] = "Total"
    df4.loc['Total','Team'] = season +1
    dft = df4.tail(1)
    dfa.append(dft)
df_mw = pd.concat(dfa).reset_index(drop=True)
df_mw = df_mw.rename(columns={"Team":"Season"})
df_mw["Win_PCT"] = (df_mw["Win_More_3PT"]/(df_mw["Win_More_3PT"]+df_mw["Loss_More_3PT"])).round(3)

In [ ]:
p = (
    ggplot(df_mw)
    + aes(x="Season", y= "Win_PCT")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "Team Win % When Making 2 More 3 Pointers than Opponent",
            caption="@SravanNBA | source:nba.com/stats",
            y = "Win %",
        )
    + theme(plot_title=element_text(face="bold", size=11.5))
)
p.save(fig_DIR + "FG3M_Wins_seasons_1.png", width=5, height=4, dpi=300)
p.draw()

# 3FG% vs Wins Year to Year

### Team 3P% > Opponent 3P%

In [ ]:
dfa = []
for season in range(2014,2024):
    df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df['Team_Split'] = df['MATCHUP'].str.replace("@","")
    df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
    df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
    df[["FG3M","PLUS_MINUS","FG3_PCT"]] = df[["FG3M","PLUS_MINUS","FG3_PCT"]].astype(str)
    df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3_PCT","PLUS_MINUS"]].agg(",".join).reset_index()
    df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df1[["FG3_PCT_For", "FG3_PCT_Against"]] = df1["FG3_PCT"].str.split(",", expand=True)
    df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
    df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3_PCT","PLUS_MINUS"]].agg(",".join).reset_index()
    df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df2[["FG3_PCT_Against","FG3_PCT_For"]] = df2["FG3_PCT"].str.split(",", expand=True)
    df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)
    df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
    df3 = df3[["Team1","Team2","FG3_PCT_For", "FG3_PCT_Against","PM1","PM2"]]
    df3.iloc[:,2:] = df3.iloc[:,2:].astype(float)
    df3["More_3PT_PCT"] = df3["FG3_PCT_For"] > (df3["FG3_PCT_Against"] + 0.00) 
    df3["Win"] = df3["PM1"] > 0
    df3["Loss"] = df3["PM1"] < 0
    df3["Win_More_3PT_PCT"] = df3["More_3PT_PCT"] & df3["Win"]
    df3["Loss_More_3PT_PCT"] = df3["More_3PT_PCT"] & df3["Loss"]
    df4 = df3.groupby("Team1")[["Win_More_3PT_PCT","Loss_More_3PT_PCT"]]\
        .agg({"Win_More_3PT_PCT":["sum"],"Loss_More_3PT_PCT":["sum"]})
    df4.columns = ["Win_More_3PT_PCT","Loss_More_3PT_PCT"]
    df4 = df4.sort_values(by="Win_More_3PT_PCT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
    df4.loc['Total']= df4.sum()
    df4.loc['Total','Team'] = "Total"
    df4.loc['Total','Team'] = season +1
    dft = df4.tail(1)
    dfa.append(dft)
df_pw = pd.concat(dfa).reset_index(drop=True)
df_pw = df_pw.rename(columns={"Team":"Season"})
df_pw["Win_PCT"] = (df_pw["Win_More_3PT_PCT"]/(df_pw["Win_More_3PT_PCT"]+df_pw["Loss_More_3PT_PCT"])).round(3)

In [ ]:
p = (
    ggplot(df_pw)
    + aes(x="Season", y= "Win_PCT")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "Team Win % When Having higher 3PT% than Opponent",
            caption="@SravanNBA | source:nba.com/stats",
            y = "Win %",
        )
    + theme(plot_title=element_text(face="bold", size=12))
)
p.save(fig_DIR + "FG3PCT_Wins_seasons.png", width=5, height=4, dpi=300)
p.draw()

### Team 3P% > Opponent 3P% + 5%

In [ ]:
dfa = []
for season in range(2014,2024):
    df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df['Team_Split'] = df['MATCHUP'].str.replace("@","")
    df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
    df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
    df[["FG3M","PLUS_MINUS","FG3_PCT"]] = df[["FG3M","PLUS_MINUS","FG3_PCT"]].astype(str)
    df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3_PCT","PLUS_MINUS"]].agg(",".join).reset_index()
    df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df1[["FG3_PCT_For", "FG3_PCT_Against"]] = df1["FG3_PCT"].str.split(",", expand=True)
    df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
    df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3_PCT","PLUS_MINUS"]].agg(",".join).reset_index()
    df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
    df2[["FG3_PCT_Against","FG3_PCT_For"]] = df2["FG3_PCT"].str.split(",", expand=True)
    df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)
    df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
    df3 = df3[["Team1","Team2","FG3_PCT_For", "FG3_PCT_Against","PM1","PM2"]]
    df3.iloc[:,2:] = df3.iloc[:,2:].astype(float)
    df3["More_3PT_PCT"] = df3["FG3_PCT_For"] > (df3["FG3_PCT_Against"] + 0.05) 
    df3["Win"] = df3["PM1"] > 0
    df3["Loss"] = df3["PM1"] < 0
    df3["Win_More_3PT_PCT"] = df3["More_3PT_PCT"] & df3["Win"]
    df3["Loss_More_3PT_PCT"] = df3["More_3PT_PCT"] & df3["Loss"]
    df4 = df3.groupby("Team1")[["Win_More_3PT_PCT","Loss_More_3PT_PCT"]]\
        .agg({"Win_More_3PT_PCT":["sum"],"Loss_More_3PT_PCT":["sum"]})
    df4.columns = ["Win_More_3PT_PCT","Loss_More_3PT_PCT"]
    df4 = df4.sort_values(by="Win_More_3PT_PCT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
    df4.loc['Total']= df4.sum()
    df4.loc['Total','Team'] = "Total"
    df4.loc['Total','Team'] = season +1
    dft = df4.tail(1)
    dfa.append(dft)
df_pw = pd.concat(dfa).reset_index(drop=True)
df_pw = df_pw.rename(columns={"Team":"Season"})
df_pw["Win_PCT"] = (df_pw["Win_More_3PT_PCT"]/(df_pw["Win_More_3PT_PCT"]+df_pw["Loss_More_3PT_PCT"])).round(3)

In [ ]:
p = (
    ggplot(df_pw)
    + aes(x="Season", y= "Win_PCT")
    + geom_line(group=1)
    + geom_point()
    + theme_idv
    + labs(
            title= "Team Win % When Having 5% higher 3PT% than Opponent",
            caption="@SravanNBA | source:nba.com/stats",
            y = "Win %",
        )
    + theme(plot_title=element_text(face="bold", size=12))
)
p.save(fig_DIR + "FG3PCT_Wins_seasons_1.png", width=5, height=4, dpi=300)
p.draw()